<a href="https://colab.research.google.com/github/KoreTane/interesting/blob/main/AutomacaoContasaPagar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import re
import pandas as pd
import pdfplumber  # Usando pdfplumber para extração de texto
from openpyxl import Workbook  # Para manipulação do Excel
from PIL import Image
from pyzbar.pyzbar import decode

# Função para normalizar números
def normalizar_numero(numero):
    return re.sub(r'\D', '', numero)

# Função para extrair informações da NFE
def extrair_nfe(caminho_pdf):
    try:
        with pdfplumber.open(caminho_pdf) as pdf:
            nfe_info = {}
            for pagina in pdf.pages:
                texto = pagina.extract_text()
                print(f"Texto extraído da NFE: {texto}")  # Debug: Mostra o texto extraído

                if texto:
                    # Captura o número da NFE próximo a várias palavras-chave
                    danfe_match = re.search(r'(?:Nº|Número|NF-e|Série|DANFE)[\s:.]*([\d.]+)', texto)
                    if danfe_match:
                        nfe_info['Nº DANFE'] = danfe_match.group(1).strip()

                    valor_total_match = re.search(r'Valor Total[:\s]*R\$ ([\d.,]+)', texto)
                    if valor_total_match:
                        nfe_info['VALOR TOTAL'] = float(valor_total_match.group(1).replace('.', '').replace(',', '.').strip())

                    # Retorna assim que encontrar todas as informações necessárias
                    if 'Nº DANFE' in nfe_info and 'VALOR TOTAL' in nfe_info:
                        return nfe_info

    except Exception as e:
        print(f"Erro ao ler o arquivo '{caminho_pdf}': {e}")

    return None  # Retorna None se não encontrar informações

# Função para extrair informações do boleto
def extrair_boleto(caminho_pdf):
    try:
        with pdfplumber.open(caminho_pdf) as pdf:
            for pagina in pdf.pages:
                texto = pagina.extract_text()
                print(f"Texto extraído do Boleto: {texto}")  # Debug: Mostra o texto extraído

                if texto:
                    # Captura o número do documento próximo a "Nº do Documento"
                    numero_documento_match = re.search(r'Nº do Documento[:\s]*([\d.-]+)', texto)
                    if numero_documento_match:
                        # Retorna somente a parte antes do '-'
                        numero_documento = numero_documento_match.group(1).strip().split('-')[0]
                        return numero_documento  # Retorna o número do documento

    except Exception as e:
        print(f"Erro ao ler o arquivo '{caminho_pdf}': {e}")

    return None  # Retorna None se não encontrar informações

# Caminho do diretório onde os PDFs estão armazenados
diretorio_pdf = r"C:\seu\diretorio\PDF"  # Ajuste conforme necessário
diretorio_boleto = r"C:\seu\diretorio\PDF"  # Ajuste conforme necessário

# Criar um novo arquivo Excel usando openpyxl
workbook = Workbook()
sheet = workbook.active
sheet.title = "Correspondências NFEs e Boletos"

# Cabeçalhos das colunas
headers = ['Nº DANFE', 'VALOR TOTAL NFE', 'Nº DO DOCUMENTO', 'VALOR DOCUMENTO']
sheet.append(headers)

#### Buscar todos os arquivos PDF no diretório de NFEs
for arquivo_nfe in os.listdir(diretorio_pdf):
    if arquivo_nfe.endswith(".pdf"):
        caminho_pdf_nfe = os.path.join(diretorio_pdf, arquivo_nfe)
        print(f"Lendo o arquivo NFE: {caminho_pdf_nfe}")

        nfe_info = extrair_nfe(caminho_pdf_nfe)

        if nfe_info and 'Nº DANFE' in nfe_info and 'VALOR TOTAL' in nfe_info:
            numero_danfe = nfe_info['Nº DANFE']
            valor_total_nfe = nfe_info['VALOR TOTAL']
            numero_danfe_normalizado = normalizar_numero(numero_danfe)

            # Verifica se existe um boleto correspondente no diretório de boletos
            for arquivo_boleto in os.listdir(diretorio_boleto):
                if arquivo_boleto.endswith(".pdf"):
                    caminho_pdf_boleto = os.path.join(diretorio_boleto, arquivo_boleto)
                    print(f"Verificando o boleto: {caminho_pdf_boleto}")

                    numero_documento = extrair_boleto(caminho_pdf_boleto)

                    if numero_documento is not None:
                        numero_documento_normalizado = normalizar_numero(numero_documento)

                        # Compara os números normalizados (considerando variações)
                        if numero_danfe_normalizado == numero_documento_normalizado:
                            print(f"Correspondência encontrada! NFE: {numero_danfe}, Valor Total: {valor_total_nfe}, Boleto: {numero_documento}")  # Printar correspondências

                            # Adiciona os dados ao Excel usando openpyxl
                            row_data = [nfe_info.get('Nº DANFE'),
                                        valor_total_nfe,
                                        numero_documento,
                                        "Valor Correspondente"]  # Adiciona uma coluna para o valor do documento se necessário.
                            sheet.append(row_data)  # Adiciona a linha ao Excel

# Salvar o arquivo Excel
output_path = r"C:\Users\Public\PDF\nfe_e_boletos.xlsx"  # Caminho específico
workbook.save(output_path)
print(f"Dados salvos em '{output_path}'.")